# MLP

<img src='http://i.stack.imgur.com/3mnuT.png' />

[Wikipedia](https://en.wikipedia.org/wiki/Backpropagation)

## Notations

### Data
- input: $ \mathbf{x} = [1, x_1, x_2, ..., x_D] $  
- class: $ \mathcal{C} = \{C_1, C_2, ..., C_{N_C}\} $  
- target output: $ \mathbf{t} = [t_1, ..., t_{N_C}] $

예를 들어, 클래스가 {'미국', '중국', '일본'} 이라면,  
'미국'은 $ \mathbf{t} = [1, 0, 0] $  
'중국'은 $ \mathbf{t} = [0, 1, 0] $  
'일본'은 $ \mathbf{t} = [0, 0, 1] $  
으로 변환한다.

따라서 학습에 사용될 데이터 $ \mathscr{D} $는 다음과 같이 표현된다.  
$ \mathscr{D} = [ [\mathbf{x}^{(1)}, \mathbf{t}^{(1)}], [\mathbf{x}^{(2)}, \mathbf{t}^{(2)}], ... , [\mathbf{x}^{(N)}, \mathbf{t}^{(N)}]  ] $  



### Neuron
For each neuron j,  

- output: $ o_j = \sigma(\mathtt{net}_j) $  
- activation function: $ \sigma(\cdot) $  
- weighted sum to neuron $j$: $\mathtt{net}_j = \sum_{i=0}^{N_i}{w_{ij}o_i}$  
- weight from neuron $i$ to neuron $j$: $w_{ij}$

### MLP

input: $ \mathbf{x} = [1, x_1, ..., x_i, ... , x_{N_i}] $  
hidden: $ \mathbf{h} = [1, h_1, ..., h_j, ... , h_{N_j}] $  
actual output: $ \mathbf{y} = [y_1, ..., y_k, ... , y_{N_k}] $  

$i$ : index for input neurons  
$j$ : index for hidden neurons  
$k$ : index for output neurons  

$h_j = \sigma(\mathtt{net}_j) = \sigma(\sum_{i=0}^{N_i}{w_{ij}x_i}) $  
$y_k = \sigma(\mathtt{net}_k) = \sigma(\sum_{j=0}^{N_j}{w_{jk}h_j}) $  
